In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

In [9]:
# checking the model
from src.model import EncOrtho, AEOrtho
import torch

# open yaml file as dictionary
import yaml
with open('./confs/model/cnn_ortho.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# erase key 'code' from dictionary
config.pop('code')

input_rand = torch.randn(100, 3, 32, 32)
model_ae = AEOrtho(config)
model_enc = EncOrtho(config)
output_ae = model_ae(input_rand)
output_enc = model_enc(input_rand)

print('model_ae: ', model_ae, 'example_output', output_ae)
print('model_enc: ', model_enc, 'example_output', output_enc)

model_ae:  AEOrtho(
  (encoder): Encoder(
    (0): ConvNet(
      (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.0001, momentum=0.1, affine=False, track_running_stats=True)
      (2): ELU(alpha=1.0)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvNet(
      (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=False, track_running_stats=True)
      (2): ELU(alpha=1.0)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvNet(
      (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (1): BatchNorm2d(128, eps=0.0001, momentum=0.1, affine=False, track_running_stats=True)
      (2): ELU(alpha=1.0)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )

In [4]:
%load_ext tensorboard # if want to show it here
%tensorboard --logdir=runs

#show the tensorboard
writer = SummaryWriter()
writer.add_graph(model_ae, input_rand)
writer.add_graph(model_enc, input_rand)
writer.close()

ModuleNotFoundError: No module named 'tensorboard # if want to show it here'

In [10]:
# load the csv file
df_raw = pd.read_csv('auc_results.csv')
df_raw.head()

,Name,_wandb,auc
0,Group: skip_do2hsc,NaN,0.494966
1,Group: skip_do2hsc,NaN,0.566402
2,Group: skip_do2hsc,NaN,0.453072
3,Group: skip_do2hsc,NaN,0.518482
4,Group: skip_do2hsc,NaN,0.462096


In [11]:
# get only columns 'Name' and 'auc'
df_auc = df_raw[['Name', 'auc']]
df_auc.head()

,Name,auc
0,Group: skip_do2hsc,0.494966
1,Group: skip_do2hsc,0.566402
2,Group: skip_do2hsc,0.453072
3,Group: skip_do2hsc,0.518482
4,Group: skip_do2hsc,0.462096


In [12]:
# get the unique values of 'Name'
df_auc['Name'].unique()

array(['Group: skip_do2hsc', 'Group: ortho_do2hsc', 'Group: base_base',
       'Group: skip_dohsc', 'Group: ortho_dohsc'], dtype=object)

In [15]:
# for each unique value of 'Name', create a new dataframe with the values of 'auc'
df_auc_0 = df_auc[df_auc['Name'] == df_auc['Name'].unique()[0]] #skip_do2hsc
df_auc_1 = df_auc[df_auc['Name'] == df_auc['Name'].unique()[1]] #ortho_do2hsc
df_auc_2 = df_auc[df_auc['Name'] == df_auc['Name'].unique()[2]] #base
df_auc_3 = df_auc[df_auc['Name'] == df_auc['Name'].unique()[3]] #skip_dohsc
df_auc_4 = df_auc[df_auc['Name'] == df_auc['Name'].unique()[4]] #ortho_dohsc

# reset the index of the dataframes
df_auc_0 = df_auc_0.reset_index(drop=True) # skip_do2hsc
df_auc_1 = df_auc_1.reset_index(drop=True) # ortho_do2hsc
df_auc_2 = df_auc_2.reset_index(drop=True) # base
df_auc_3 = df_auc_3.reset_index(drop=True) # skip_dohsc
df_auc_4 = df_auc_4.reset_index(drop=True) # ortho_dohsc

df_long = pd.concat([df_auc_2, df_auc_4, df_auc_1,df_auc_3,df_auc_0], axis=1)
# drop columns 'Name'
df_long = df_long.drop(columns=['Name'])
# rename columns
df_long.columns = ['auc_base', 'auc_ortho_dohsc', 'auc_ortho_do2hsc', 'auc_skip_dohsc', 'auc_skip_do2hsc']
# label of CIFAR-10
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# assign labels to the index
df_long.index = labels
df_long

,auc_base,auc_ortho_dohsc,auc_ortho_do2hsc,auc_skip_dohsc,auc_skip_do2hsc
airplane,0.625332,0.807429,0.550922,0.778600,0.494966
automobile,0.498241,0.778136,0.413177,0.798894,0.566402
bird,0.506461,0.731340,0.501813,0.705305,0.453072
cat,0.522005,0.767414,0.509876,0.825998,0.518482
deer,0.443229,0.677868,0.595395,0.710667,0.462096
dog,0.503729,0.536661,0.473181,0.653098,0.475568
frog,0.465518,0.619313,0.489470,0.622623,0.523069
horse,0.491382,0.619679,0.509680,0.634519,0.444832
ship,0.564833,0.703277,0.467833,0.766374,0.449965
truck,0.520972,0.772023,0.538794,0.773195,0.391118


In [16]:
# write the dataframe as latex show it here
print(df_long.style.highlight_max(axis=1, props="textbf:--rwrap;").to_latex())

\begin{tabular}{lrrrrr}
 & auc_base & auc_ortho_dohsc & auc_ortho_do2hsc & auc_skip_dohsc & auc_skip_do2hsc \\
airplane & 0.625332 & \textbf{0.807429} & 0.550922 & 0.778600 & 0.494966 \\
automobile & 0.498241 & 0.778136 & 0.413177 & \textbf{0.798894} & 0.566402 \\
bird & 0.506461 & \textbf{0.731340} & 0.501813 & 0.705305 & 0.453072 \\
cat & 0.522005 & 0.767414 & 0.509876 & \textbf{0.825998} & 0.518482 \\
deer & 0.443229 & 0.677868 & 0.595395 & \textbf{0.710667} & 0.462096 \\
dog & 0.503729 & 0.536661 & 0.473181 & \textbf{0.653098} & 0.475568 \\
frog & 0.465518 & 0.619313 & 0.489470 & \textbf{0.622623} & 0.523069 \\
horse & 0.491382 & 0.619679 & 0.509680 & \textbf{0.634519} & 0.444832 \\
ship & 0.564833 & 0.703277 & 0.467833 & \textbf{0.766374} & 0.449965 \\
truck & 0.520972 & 0.772023 & 0.538794 & \textbf{0.773195} & 0.391118 \\
\end{tabular}

